## Clasificación de textos utilizando AutoML


La clasificación de textos consiste en, dado un texto, asignarle una entre varias categorías. Algunos ejemplos de esta tarea son:

- dado un tweet, categorizar su connotación como positiva, negativa o neutra.
- dado un post de Facebook, clasificarlo como portador de un lenguaje ofensivo o no.  

En la actividad exploraremos cómo utilizar soluciones *out of the box* para esta tarea incluidas en la librería [AutoGOAL](https://github.com/autogoal/autogoal) y su aplicación para clasificar reviews de [IMDB](https://www.imdb.com/) sobre películas en las categorías \[$positive$, $negative$\]. 



**Instrucciones:**

- siga las indicaciones y comentarios en cada apartado.


**Después de esta actividad nos habremos familiarizado con:**
- cómo modelar un problema de clasificación con AutoGOAL
- cómo utilizar AutoGOAL para buscar automáticamente un *pipeline* para clasificación de textos.
- utilizar este *pipeline* para clasificar nuevos textos.

**Requerimientos**
- python 3.6.12 - 3.8
- tensorflow==2.3.0
- autogoal==0.3.2
- pandas==1.1.5
- plotly==4.13.0
- tqdm==4.56.0


<a name="setup"></a>
### Instalación de librerías e importación de dependencias.

Para comenzar, es preciso instalar e incluir las librerías necesarias. En este caso, el entorno de Colab incluye las necesarias.

Ejecute la siguiente casilla prestando atención a las explicaciones dadas en los comentarios.

In [3]:
# instalar librerías. Esta casilla es últil por ejemplo si se ejecuta el cuaderno en Google Colab
# Note que existen otras dependencias como tensorflow, etc. que en este caso se encontrarían ya instaladas
%%capture
!pip install autogoal[contrib]==0.3.2

print('Done!')

In [19]:
# reset environment
#%reset -f

#  para construir gráficas y realizar análisis exploratorio de los datos
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

# para cargar datos y realizar pre-procesamiento básico
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

# para evaluar los modelos 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# para configurar AutoGOAL
from autogoal.ml import AutoML
from autogoal.search import (Logger, PESearch, ConsoleLogger, ProgressLogger, MemoryLogger,
)
from autogoal.kb import MatrixContinuousDense, CategoricalVector
from autogoal.contrib import find_classes

# para guardar el modelo
import pickle
import datetime

print('Done!')


Done!


#### Definición de funciones y variables necesarias para el pre-procesamiento de datos

Antes de definir el pipeline definiremos algunas variables útiles como el listado de stop words y funciones para cargar los datos, entrenar el modelo etc.

In [5]:
# función auxiliar que evalúa los resultados de una clasificación
def evaluate_model(y_true, y_pred, y_score=None, pos_label='positive'):
  """
  data: list of the text to predict
  pref: identificador para las columnas (labels_[pref], scores_[pref]_[class 1], etc.)
  """
  print('==== Sumario de la clasificación ==== ')
  print(classification_report(y_true, y_pred))

  print('Accuracy -> {:.2%}\n'.format(accuracy_score(y_true, y_pred)))

  # graficar matriz de confusión
  display_labels = sorted(unique_labels(y_true, y_pred), reverse=True)
  cm = confusion_matrix(y_true, y_pred, labels=display_labels)

  z = cm[::-1]
  x = display_labels
  y =  x[::-1].copy()
  z_text = [[str(y) for y in x] for x in z]

  fig_cm = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

  fig_cm.update_layout(
      height=400, width=400,
      showlegend=True,
      margin={'t':150, 'l':0},
      title={'text' : 'Matriz de Confusión', 'x':0.5, 'xanchor': 'center'},
      xaxis = {'title_text':'Valor Real', 'tickangle':45, 'side':'top'},
      yaxis = {'title_text':'Valor Predicho', 'tickmode':'linear'},
  )
  fig_cm.show()


  # curva roc (definido para clasificación binaria)
  fig_roc = None
  if y_score is not None:
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=pos_label)
    fig_roc = px.area(
        x=fpr, y=tpr,
        title={'text' : f'Curva ROC (AUC={auc(fpr, tpr):.4f})', 'x':0.5, 'xanchor': 'center'},
        labels=dict(x='Ratio Falsos Positivos', y='Ratio Verdaderos Positivos'),
        width=400, height=400
    )
    fig_roc.add_shape(type='line', line=dict(dash='dash'), x0=0, x1=1, y0=0, y1=1)

    fig_roc.update_yaxes(scaleanchor="x", scaleratio=1)
    fig_roc.update_xaxes(constrain='domain')
    
    fig_roc.show()


# Custom logger
# - imprime y guarda el mejor pipeline cada vez que se encuentre una nueva solución candidad
# - imprime pipelines cuya evaluación falló
class CustomLogger(Logger):
    def __init__(self, classifier, save_model=True, check_folder="."):
        self.save_model = save_model
        self.check_folder = check_folder
        self.classifier = classifier

    def error(self, e: Exception, solution):
        if e and solution:
            with open("reviews_errors.log", "a") as fp:
                fp.write(f"solution={repr(solution)}\nerror={repr(e)}\n\n")

    def update_best(self, new_best, new_fn, *args):
        pipecode = datetime.datetime.now(datetime.timezone.utc).strftime("reviews--%Y-%m-%d--%H-%M-%S--{0}".format(hex(id(new_best))))
        with open("reviews_update_best.log", "a") as fp:
            fp.write(f"\n{pipecode}\nsolution={repr(new_best)}\nfitness={new_fn}\n\n")

        if(self.save_model):
            fp = open('{1}.pickle'.format(self.check_folder,pipecode), 'wb')
            new_best.sampler_.replay().save(fp)
            pickle.Pickler(fp).dump((self.classifier.input, self.classifier.output))
            fp.close()

print('Done!')

Done!


### Carga de datos y análisis exploratorio

Antes de entrenar el pipeline, es necesario cargar los datos. Existen diferentes opciones, entre estas:

- montar nuestra partición de Google Drive y leer un fichero desde esta.

- leer los datos desde un fichero en una carpeta local.

- leer los datos directamente de un URL.

Ejecute la siguiente casilla prestando atención a las instrucciones adicionales en los comentarios.


In [6]:
# descomente las siguientes 3 líneas para leer datos desde Google Drive, asumiendo que se trata de un fichero llamado review.csv localizado dentro de una carpeta llamada 'Datos' en su Google Drive
#from google.colab import drive
#drive.mount('/content/drive')
#path = '/content/drive/MyDrive/Datos/ejemplo_review_train.csv'

# descomente la siguiente línea para leer los datos desde un archivo local, por ejemplo, asumiendo que se encuentra dentro de un directorio llamado sample_data
#path = './sample_data/ejemplo_review_train.csv'

# descomente la siguiente línea para leer datos desde un URL
path = 'https://github.com/TeachingTextMining/TextClassification/raw/main/06-SA-AutoGOAL/sample_data/ejemplo_review_train.csv'

# leer los datos
data = pd.read_csv(path, sep=',')

print('Done!')

Done!


Una vez leídos los datos, ejecute la siguiente casilla para construir una gráfica que muestra la distribución de clases en el corpus. 

In [7]:
text_col = 'Phrase'  # columna del dataframe que contiene el texto (depende del formato de los datos)
class_col = 'Sentiment'  # columna del dataframe que contiene la clase (depende del formato de los datos)

# obtener algunas estadísticas sobre los datos
categories = sorted(data[class_col].unique(), reverse=True)
hist= Counter(data[class_col]) 
print(f'Total de instancias -> {data.shape[0]}')
print(f'Distribución de clases -> {{item[0]:round(item[1]/len(data[class_col]), 3) for item in sorted(hist.items(), key=lambda x: x[0])}}')

print(f'Categorías -> {categories}')
print(f'Comentario de ejemplo -> {data[text_col][0]}')
print(f'Categoría del comentario -> {data[class_col][0]}')

fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in sorted(hist.keys())]))
fig.show()

print('Done!')

Total de instancias -> 1763
Distribución de clases -> {item[0]:round(item[1]/len(data[class_col]), 3) for item in sorted(hist.items(), key=lambda x: x[0])}
Categorías -> ['positive', 'negative']
Comentario de ejemplo -> This is a great movie that everyone should see. It plays like a Dean Koontz book.<br /><br />Bill Paxton's performance was great in that it really seems like he believes in what he is saying and doing.<br /><br />I don't know why viewers have to read in some kind of advocacy for religious murder in to the film. It is fiction. The ending is surprising, but fictional. So what? I think that is what makes this movie so good. SPOILER DO NOT READ FURTHER IF YOU HAVENT SEEN THE MOVIE. Throughout the movie, the viewer is continually shocked at the sickness of Paxton's character, the impact on the children, and the way the children handle this outrageous conduct. And then at the end, it turns out to be true. God has put him on a mission to rid the world of demons. Paxton is not 

Done!


Finalmente, ejecute la siguiente casilla para crear los conjuntos de entrenamiento y validación que se utilizarán para entrenar y validar los modelos.

In [8]:
# obtener conjuntos de entrenamiento (90%) y validación (10%)
seed = 0  # fijar random_state para reproducibilidad
train, val = train_test_split(data, test_size=.1, stratify=data[class_col], random_state=seed)

print('Done!')

Done!


### Implementación y configuración del modelo

Con AutoGOAL podemos configurar el modelo facilmente pues solo necesitamos instanciar la clase AutomML. Lo más importante es elegir los tipos adecuados para datos de entrada y salida en nuestro modelo y la métrica de evaluación. En este caso:

- entrada (input): MatrixContinuousDense -> una fila por instancia y una columna por variable.

- salida (output): CategoricalVector -> el elemento *i* representa la categoría asociada a la instancia *i*.

Ejecute la siguiente casilla prestando atención a los comentarios adicionales.

In [15]:
# configuraciones
cfg = {}
cfg['iterations'] = 1 # cantidad de iteraciones a realizar
cfg['popsize'] = 50  # tamaño de la población
cfg['search_timeout'] = 3600  # tiempo máximo de búsqueda en segundos
cfg['evaluation_timeout'] = 600  # tiempo máximo que empleará evaluando un pipeline en segundos
cfg['memory'] = 20  # cantidad máxima de memoria a utilizar
cfg['score_metric'] = f1_score  # métrica de evaluación

classifier = AutoML(
    input=MatrixContinuousDense(),  # tipo datos de entrada
    output=CategoricalVector(),  # tipo datos de salida
    
    score_metric=cfg['score_metric'],
    search_algorithm=PESearch,  # algoritmo de búsqueda
    registry=None,  # para incluir clases adicionales
    
    search_kwargs=dict(
        pop_size=cfg['popsize'],
        search_timeout=cfg['search_timeout'],
        evaluation_timeout=cfg['evaluation_timeout'],
        memory_limit=cfg['memory'] * 1024 ** 3,
    ), 
    search_iterations=cfg['iterations'],
    
    include_filter=".*",  # indica qué módulos pueden incluirse en los pipelines evaluados
    exclude_filter=None,  # indica módulos a excluir de los pipelines evaluados
    
    validation_split=0.3,  # porción de los datos de entrenamiento que AutoGOAL tomará para evaluar cada pipeline
    cross_validation_steps=3,  # cantidad de particiones en la crossvalidación
    cross_validation="mean",  # tipo de agregación para los valores de la métrica en cada partición de la crossvalidación (promedio, mediana, etc.)
    
    random_state=None,  # semilla para el generador de números aleatorios
    errors="warn",  # tratamiento ante errores
    metalearning_log=False,  # logs adicionales de la librería AutoGOAL
)

print('Done!')

Done!


### Pre-procesamiento de los datos

Antes de entrenar, debemos pre-procesar los datos. Esto dependerá de la tarea en particular, en este caso, comprende:

- obtener vectores tf-idf correspondientes a cada ejemplo.

**Notar que:**

- en dependencia de su implementación del extractor, deberá re-implementar esta sección.

Ejecute la siguiente casilla prestando atención a los comentarios explicativos.

#### Instanciar tf-idf vectorizer

In [21]:
# instanciar TfidfVectorizer
cfg['vectorizer'] = TfidfVectorizer(stop_words='english', max_features=10000)

print('Done!')

Done!


#### Pre-procesamiento

In [22]:
# entrenar TfidfVectorizer
cfg['vectorizer'].fit(train[text_col].to_list())

# guardar TfidfVectorizer entrenado para su posterior uso (codificar nuevos datos).
with open('vectorizer_reviews.pkl', 'wb') as f:
    pickle.dump(cfg['vectorizer'], f)

print('Done!')

Done!


### Entrenamiento del modelo

Por último es necesario "entrenar el modelo", que en este caso significa iniciar la búsqueda.

classifier.fit(X_train, y_train, logger=loggers)